<a href="https://colab.research.google.com/github/abakamousa/NER-Sangkak-challenge/blob/0.1/training/experimentations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install librairies

In [2]:
!pip uninstall typing -y
!pip install flair==0.9

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.3/319.3 KB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 103.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 KB 52.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 KB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#!pip install flair
#!pip uninstall flair


#Import librairies

In [15]:

import pandas as pd

from flair.data import Corpus
from flair.datasets import ColumnCorpus


import flair
from typing import List
from flair.trainers import ModelTrainer
from flair.models import SequenceTagger
from flair.embeddings import TransformerWordEmbeddings
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, FlairEmbeddings


In [4]:
# define columns
columns = {0 : 'text', 1 : 'ner'}# directory where the data resides
data_folder = '/content/bbj/'# initializing the corpus
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file = 'train.txt',
                              test_file = 'test.txt',
                              dev_file = 'dev.txt')

2023-03-02 14:27:13,140 Reading data from /content/bbj
2023-03-02 14:27:13,146 Train: /content/bbj/train.txt
2023-03-02 14:27:13,154 Dev: /content/bbj/dev.txt
2023-03-02 14:27:13,160 Test: /content/bbj/test.txt


In [4]:
print(len(corpus.train))

3384


In [5]:
print(corpus.train[0].to_tagged_string('ner'))

Msaʼnyə̂ gɔtí cyətə nə́ bǎyá cyə́ nəjí pôʼ bǎhə́lə́


In [5]:
tag_type = 'ner'

#  make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)

In [6]:
embedding_types = [

    WordEmbeddings('glove'),

    # comment in this line to use character embeddings
    # CharacterEmbeddings(),

    # comment in these lines to use flair embeddings
    # FlairEmbeddings('news-forward'),
    # FlairEmbeddings('news-backward'),
]

2023-03-02 14:27:14,934 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim.vectors.npy not found in cache, downloading to /tmp/tmpn8f0ouhq


100%|██████████| 160000128/160000128 [00:03<00:00, 40101099.72B/s]

2023-03-02 14:27:19,006 copying /tmp/tmpn8f0ouhq to cache at /root/.flair/embeddings/glove.gensim.vectors.npy


2023-03-02 14:27:19,281 removing temp file /tmp/tmpn8f0ouhq
2023-03-02 14:27:19,397 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim not found in cache, downloading to /tmp/tmpbxm2qdwv


100%|██████████| 21494764/21494764 [00:00<00:00, 22566622.39B/s]

2023-03-02 14:27:20,449 copying /tmp/tmpbxm2qdwv to cache at /root/.flair/embeddings/glove.gensim
2023-03-02 14:27:20,469 removing temp file /tmp/tmpbxm2qdwv


In [16]:
embeddings = StackedEmbeddings(embeddings=embedding_types)

# initialize sequence tagger
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=tag_dictionary,
                        tag_type=tag_type,
                        use_crf=True)

# initialize trainer
trainer = ModelTrainer(tagger, corpus)





In [17]:
embeddings_transf = TransformerWordEmbeddings(model='xlm-roberta-large',
                                       layers="-1",
                                       subtoken_pooling="first",
                                       fine_tune=True,
                                       use_context=True,
                                       )

# initialize sequence tagger
tagger_transf = SequenceTagger(hidden_size=256,
                        embeddings=embeddings_transf,
                        tag_dictionary=tag_dictionary,
                        tag_type=tag_type,
                        use_crf=True)

trainer1 = ModelTrainer(tagger, corpus)

In [10]:
# start training with ner-multi-fast'
trainer.train('resources/taggers/ner-multi-fast',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=10)

2023-03-02 14:35:24,752 ----------------------------------------------------------------------------------------------------
2023-03-02 14:35:24,755 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('glove')
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=100, out_features=100, bias=True)
  (rnn): LSTM(100, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=11, bias=True)
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)"
2023-03-02 14:35:24,757 ----------------------------------------------------------------------------------------------------
2023-03-02 14:35:24,759 Corpus: "Corpus: 3384 train + 483 dev + 966 test sentences"
2023-03-02 14:35:24,762 ----------------------------------------------------------------------------------------------------
2023-03-02 14:35:24,764 Parameters:
2023-03-02 14:35:24,766  - learning_

{'test_score': 0.3468286099865047,
 'dev_score_history': [],
 'train_loss_history': [],
 'dev_loss_history': []}

In [18]:
# start training with xlm-roberta-large-masakhaner

trainer1.train('resources/taggers/xlm-roberta-large-masakhaner',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=10)

2023-03-02 15:32:03,130 ----------------------------------------------------------------------------------------------------
2023-03-02 15:32:03,135 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('glove')
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=100, out_features=100, bias=True)
  (rnn): LSTM(100, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=11, bias=True)
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)"
2023-03-02 15:32:03,137 ----------------------------------------------------------------------------------------------------
2023-03-02 15:32:03,140 Corpus: "Corpus: 3384 train + 483 dev + 966 test sentences"
2023-03-02 15:32:03,143 ----------------------------------------------------------------------------------------------------
2023-03-02 15:32:03,147 Parameters:
2023-03-02 15:32:03,150  - learning_

{'test_score': 0.33617859487852925,
 'dev_score_history': [0.12529002320185614,
  0.13170731707317074,
  0.13106796116504857,
  0.17937219730941703,
  0.17155756207674944,
  0.16052060737527116,
  0.175054704595186,
  0.18984547461368653,
  0.23030303030303031,
  0.19108280254777066],
 'train_loss_history': [0.48352091876519554,
  0.33815440963221516,
  0.30448860584012655,
  0.2889537602532447,
  0.27726423012851703,
  0.2738213424151262,
  0.26819249112377064,
  0.2675267200586659,
  0.2647055175292855,
  0.2595797017314154],
 'dev_loss_history': [tensor(0.2976),
  tensor(0.2786),
  tensor(0.2548),
  tensor(0.2428),
  tensor(0.2379),
  tensor(0.2309),
  tensor(0.2376),
  tensor(0.2297),
  tensor(0.2362),
  tensor(0.2229)]}